## Licence
Copyright (c) 2023, RTE (http://www.rte-france.com)  
This Source Code Form is subject to the terms of the Mozilla Public  
License, v. 2.0. If a copy of the MPL was not distributed with this  
file, You can obtain one at http://mozilla.org/MPL/2.0/.
SPDX-License-Identifier: MPL-2.0

# Network modifications 

This notebook explains how to easily create a network in Node/Breaker topology and how to modify network elements.
In the Node/Breaker topology, all the switches are represented and each equipment is associated with a node. It is the most detailed representation of a network. 
The methods shown in this notebook can be used to create a network from scratch or to modify an existing network after it has been imported. They allow to easily create voltage levels and equipment with their bay i.e. with the associated breakers and disconnectors in one line.
We will only need two libraries: pypowsybl and pandas, to deal with dataframes.

In [1]:
import pypowsybl as pp
import pandas as pd

First we can create an empty network using the pypowsybl function.

In [2]:
n = pp.network.create_empty()

## How to create voltage levels 
Then we create two substations S1 and S2 and their voltage levels. S1 will have two voltage levels, VL1 and VL2, and S2 will have one voltage level VL3. To create the voltage levels, we must specify the ID, as well as the type of topology, in this case Node/Breaker, and the nominal voltage. We can put all this data in a dataframe and use the dataframe as an argument to the create_voltage_levels function.

In [3]:
n.create_substations(id=['S1', 'S2'])

In [4]:
voltage_levels = pd.DataFrame.from_records(index='id', data=[
    {'substation_id': 'S1', 'id': 'VL1', 'topology_kind': 'NODE_BREAKER', 'nominal_v': 400},
    {'substation_id': 'S1', 'id': 'VL2', 'topology_kind': 'NODE_BREAKER', 'nominal_v': 225},
    {'substation_id': 'S2', 'id': 'VL3', 'topology_kind': 'NODE_BREAKER', 'nominal_v': 400},
])
n.create_voltage_levels(voltage_levels)

The voltage levels we have just created are completely empty. We will now create the topology of each voltage level, i.e. the busbar sections and the switches that make up the structure of the voltage level. The function used only works to create symmetrical voltage levels, with a given number of busbar and the same switches between each section on a same busbar. The switches should be given as a list. 

In [5]:
voltage_level_topology = pd.DataFrame.from_records(index='id', data=[
    {'id': 'VL1', 'aligned_buses_or_busbar_count': 1, 'switch_kinds': ''},
    {'id': 'VL2', 'aligned_buses_or_busbar_count': 3, 'switch_kinds': 'BREAKER, DISCONNECTOR'},
    {'id': 'VL3', 'aligned_buses_or_busbar_count': 2, 'switch_kinds': 'DISCONNECTOR'},
])
pp.network.create_voltage_level_topology(n, voltage_level_topology)

With a single line diagram, we can check that the topology is correct.

In [6]:
n.get_single_line_diagram('S1')

In [7]:
n.get_single_line_diagram('S2')

## How to create coupling devices
Coupling devices can also be created. The function takes as input the two busbar sections to be connected with a closed disconnector. An open disconnector is created on each parallel busbar on the same section. Only the IDs of the busbar sections where the disconnector is to be closed need to be specified.

NB: If the coupling device is created on a voltage level with only two busbar sections and between busbar sections with the same index, i.e. not around a switch, then only two closed disconnectors will be created as all possible coupling possibilities are then available. In our example, this is the case on the voltage level VL3.

In [8]:
coupling_device = pd.DataFrame.from_records(index='bus_or_busbar_section_id_1', data=[
    {'bus_or_busbar_section_id_1': 'VL2_1_1', 'bus_or_busbar_section_id_2': 'VL2_2_1'},
    {'bus_or_busbar_section_id_1': 'VL2_1_1', 'bus_or_busbar_section_id_2': 'VL2_1_2'},
    {'bus_or_busbar_section_id_1': 'VL2_1_2', 'bus_or_busbar_section_id_2': 'VL2_2_2'},
    {'bus_or_busbar_section_id_1': 'VL2_1_3', 'bus_or_busbar_section_id_2': 'VL2_2_3'},
    {'bus_or_busbar_section_id_1': 'VL3_1_1', 'bus_or_busbar_section_id_2': 'VL3_2_1'},
    {'bus_or_busbar_section_id_1': 'VL3_1_2', 'bus_or_busbar_section_id_2': 'VL3_2_2'},
])
pp.network.create_coupling_device(n, coupling_device)

In [9]:
n.get_single_line_diagram('S1')

In [10]:
n.get_single_line_diagram('S2')

## How to create injections
Now let's create all the injections in the network. Pypowsybl has functions to easily create these elements with their topology. They are added with their bay: a breaker is automatically created between the feeder and the busbar section to which it is connected by a closed disconnector. Open disconnectors are also created on each parallel busbar section.
It is necessary to specify as input the characteristics of the injection, e.g. p0 and q0 for a load, as well as the ID of the busbar section on which the disconnector is to be closed and the position of the injection. This position allows the ConnectablePosition extension to be filled and the injections to be positioned relative to each other. It is also used to display the single line diagram. Optionally, the direction of the injection can also be specified for the single line diagram and is set to 'BOTTOM' by default.

#### Adding loads

In [11]:
loads = pd.DataFrame.from_records(
        index='id',
        columns=['id', 'p0', 'q0', 'bus_or_busbar_section_id', 'position_order'],
        data=[('load1', 10.0, 3.0, 'VL1_1_1', 10),
              ('load2', 10.0, 3.0, 'VL2_1_3', 60),
              ('load3', 10.0, 3.0, 'VL2_2_3', 70),
              ('load4', 10.0, 3.0, 'VL2_3_3', 80),
              ('load5', 10.0, 3.0, 'VL3_1_1', 10)])
loads

,p0,q0,bus_or_busbar_section_id,position_order
id,,,,
load1,10.0,3.0,VL1_1_1,10
load2,10.0,3.0,VL2_1_3,60
load3,10.0,3.0,VL2_2_3,70
load4,10.0,3.0,VL2_3_3,80
load5,10.0,3.0,VL3_1_1,10


In [12]:
pp.network.create_load_bay(n, loads)

In [13]:
n.get_single_line_diagram('S1')

In [14]:
n.get_single_line_diagram('S2')

#### Adding generators

In [15]:
generators = pd.DataFrame.from_records(
        index='id',
        columns=['id', 'max_p', 'min_p', 'voltage_regulator_on', 'target_p', 'target_q', 'target_v', 'bus_or_busbar_section_id', 'position_order', 'direction'],
        data=[('gen1', 1000.0, 0, True, 100.0, 150.0, 225.0, 'VL2_1_2', 30, 'TOP'),
              ('gen2', 1000.0, 0, True, 100.0, 150.0, 225.0, 'VL2_1_2', 40, 'TOP'),
              ('gen3', 1000.0, 0, True, 100.0, 150.0, 225.0, 'VL2_1_2', 50, 'TOP')])
generators

,max_p,min_p,voltage_regulator_on,target_p,target_q,target_v,bus_or_busbar_section_id,position_order,direction
id,,,,,,,,,
gen1,1000.0,0,True,100.0,150.0,225.0,VL2_1_2,30,TOP
gen2,1000.0,0,True,100.0,150.0,225.0,VL2_1_2,40,TOP
gen3,1000.0,0,True,100.0,150.0,225.0,VL2_1_2,50,TOP


In [16]:
pp.network.create_generator_bay(n, generators)

In [17]:
n.get_single_line_diagram('S1')

#### Adding a shunt
To add a shunt, two dataframes must be given: one describing the shunt and where/how it should be connected and one describing its model.

In [18]:
shunt_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'model_type', 'section_count', 'target_v',
             'target_deadband', 'bus_or_busbar_section_id', 'position_order', 'direction'],
    data=[('shunt1', 'LINEAR', 1, 221, 2, 'VL2_3_3', 90, 'TOP')])
model_df = pd.DataFrame.from_records(
    index='id',
    columns=['id', 'g_per_section', 'b_per_section', 'max_section_count'],
    data=[('shunt1', 0.014, 0.0001, 2)])
shunt_df

,model_type,section_count,target_v,target_deadband,bus_or_busbar_section_id,position_order,direction
id,,,,,,,
shunt1,LINEAR,1,221,2,VL2_3_3,90,TOP


In [19]:
pp.network.create_shunt_compensator_bay(n, shunt_df=shunt_df, linear_model_df=model_df)

In [20]:
n.get_single_line_diagram('S1')

#### Adding a SVC

In [21]:
pp.network.create_static_var_compensator_bay(n, id='svc', target_q=200.0, regulation_mode='REACTIVE_POWER', target_v=400, b_min=0, b_max=2, bus_or_busbar_section_id='VL3_1_1', position_order=20, direction='TOP')

In [22]:
n.get_single_line_diagram('VL3')

## How to create branches
To create lines and two-winding transformers, the method is very similar to creating injections, but with the specification of the busbar section ID, the position and the direction on both sides. Note that the default direction for branches is TOP.

In [23]:
pp.network.create_2_windings_transformer_bays(n, id='twt', b=1e-6, g=1e-6, r=0.5, x=10, rated_u1=225, rated_u2=63,
            bus_or_busbar_section_id_1='VL1_1_1', position_order_1=20,
            bus_or_busbar_section_id_2='VL2_1_1', position_order_2=10)

In [24]:
pp.network.create_line_bays(n, id='line1', r=0.1, x=10, g1=0, b1=0, g2=0, b2=0, bus_or_busbar_section_id_1='VL2_3_1', position_order_1=20, direction_1='BOTTOM',
                                        bus_or_busbar_section_id_2='VL3_1_2', position_order_2=30)

In [25]:
n.get_single_line_diagram('S1')

In [26]:
n.get_single_line_diagram('S2')

## How to get the order position on existing networks
In Node/Breaker, the position of the element is mandatory. To get the available positions on a given busbar section, some built-in functions can be used.

#### Getting all the taken positions of a voltage level
The function get_connectables_order_positions returns a dataframe containing the id of each connectable of a voltage level, its order position and the extension name as displayed on the single line diagram. It takes the network and the ID of the voltage level as arguments.

In [27]:
pp.network.get_connectables_order_positions(n, 'VL2')

,order_position,extension_name
connectable_id,,
twt,10,twt
line1,20,line1
gen1,30,gen1
gen2,40,gen2
gen3,50,gen3
load2,60,load2
load3,70,load3
load4,80,load4
shunt1,90,shunt1


With this dataframe, it is possible to decide for position between two existing connectables.
#### Getting the available order positions on the right and on the left of a busbar section
Two functions exist returning the available positions at the beginning or at the end of a busbar section, in regard of other busbar sections. It returns an interval that is empty if there is no space anymore.

In [28]:
pp.network.get_unused_order_positions_before(n, 'VL3_1_1')

Interval(0, 9, closed='both')

In [29]:
pp.network.get_unused_order_positions_after(n, 'VL3_1_1')

Interval(21, 29, closed='both')

NB: All these functions to create network elements are also available for Bus/Breaker voltage levels. In this case, the ID of the busbar sections must be replaced by the ID of a bus and the positions and direction are not needed. 